## 0. Libarary 불러오기 및 경로설정

`!pip install -U albumentations==0.5.2`

`!pip install timm`

`!pip install facenet_pytorch`

In [3]:
import os
import glob
import cv2
import pandas as pd
import numpy as np
import time
import datetime
import albumentations
import timm
import wandb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision

from PIL import Image
from copy import deepcopy
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score
from pytz import timezone
from albumentations.pytorch.transforms import ToTensorV2
from facenet_pytorch import MTCNN

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim import lr_scheduler

# mtcnn 활용법 체크

```python
mtcnn = MTCNN(keep_all=True, device=cfg.device)

idx = np.random.randint(0, 18900 / 7, 1)[0]
img_path = glob.glob(cfg.img_dir + '/*/incorrect_*')[idx]
img = np.array(Image.open(img_path))
boxes, probs = mtcnn.detect(img)
if isinstance(boxes, np.ndarray):
    xmin = np.clip(int(boxes[0, 0]) - 50, 0, img.shape[0])
    ymin = np.clip(int(boxes[0, 1]) - 50, 0, img.shape[1])
    xmax = np.clip(int(boxes[0, 2]) + 50, 0, img.shape[0])
    ymax = np.clip(int(boxes[0, 3]) + 50, 0, img.shape[1])

    xmin, ymin, xmax, ymax
    img = img[ymin:ymax, xmin:xmax, :]
else:
    # 직접 crop
    img = img[100:400, 50:350, :]
print(img.shape, probs)
plt.imshow(img)
plt.show()

xmin = np.clip(int(boxes[0, 0]) - 30, 0, img.shape[0])
ymin = np.clip(int(boxes[0, 1]) - 30, 0, img.shape[1])
xmax = np.clip(int(boxes[0, 2]) + 30, 0, img.shape[0])
ymax = np.clip(int(boxes[0, 3]) + 30, 0, img.shape[1])

xmin, ymin, xmax, ymax
img = img[ymin:ymax, xmin:xmax, :]
plt.imshow(img)
plt.show()
```

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device=device)
new_img_dir = '/opt/ml/input/data/train/new_imgs'
img_path = '/opt/ml/input/data/train/images'

In [5]:
cnt = 0

for paths in tqdm(os.listdir(img_path)):
    if paths[0] == '.':
        continue
    
    sub_dir = os.path.join(img_path, paths)
    
    for imgs in os.listdir(sub_dir):
        if imgs[0] == '.':
            continue
            
        img_dir = os.path.join(sub_dir, imgs)
        img = np.array(Image.open(img_dir))
        
        # mtcnn 적용
        boxes, probs = mtcnn.detect(img)
        
        # boxes 확인
        if not isinstance(boxes, np.ndarray):
            # 직접 crop
            img = img[100:400, 50:350, :]
            cv2.resize(img, (224, 224))
        # boxes size 확인
        else:
            xmin = int(boxes[0, 0])-30
            ymin = int(boxes[0, 1])-30
            xmax = int(boxes[0, 2])+30
            ymax = int(boxes[0, 3])+30
            
            if xmin < 0: xmin = 0
            if ymin < 0: ymin = 0
            if xmax > 384: xmax = 384
            if ymax > 512: ymax = 512
            
            img = img[ymin:ymax, xmin:xmax, :]
            img = cv2.resize(img, (224, 224))
            
        tmp = os.path.join(new_img_dir, paths)
        if not os.path.exists(tmp):
            os.makedirs(tmp, exist_ok=True)
        cnt += 1
        plt.imsave(os.path.join(tmp, imgs.split('.')[0] + '.jpg'), img)
        
print(cnt)


18900


In [6]:
new_test_img_dir = '/opt/ml/input/data/eval/new_imgs'
test_img_path = '/opt/ml/input/data/eval/images'

In [7]:
cnt = 0

for imgs in tqdm(os.listdir(test_img_path)):
    if imgs[0] == '.':
        continue
        
    img_dir = os.path.join(test_img_path, imgs)
    img = np.array(Image.open(img_dir))
        
    # mtcnn 적용
    boxes, probs = mtcnn.detect(img)

    # boxes 확인
    if not isinstance(boxes, np.ndarray):
        # 직접 crop
        img = img[100:400, 50:350, :]
        cv2.resize(img, (224, 224))
        
    # boxes size 확인
    else:
        xmin = int(boxes[0, 0])-30
        ymin = int(boxes[0, 1])-30
        xmax = int(boxes[0, 2])+30
        ymax = int(boxes[0, 3])+30

        if xmin < 0: xmin = 0
        if ymin < 0: ymin = 0
        if xmax > 384: xmax = 384
        if ymax > 512: ymax = 512

        img = img[ymin:ymax, xmin:xmax, :]
        cv2.resize(img, (224, 224))
            
    if not os.path.exists(new_test_img_dir):
        os.makedirs(new_test_img_dir, exist_ok=True)
    cnt += 1
    plt.imsave(os.path.join(new_test_img_dir, imgs.split('.')[0] + '.jpg'), img)
        
print(cnt)


12600
